In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
sys.path.append('../../Shared')
import processing, Model, Visualization, modules, Adaptation
from sklearn.model_selection import train_test_split

def get_label_from_filename(fname):
    for cls, label in class_to_label.items():
        if cls in fname:
            return label
    return None  # if no class found

def get_data_all_at_once(SUB, data_lst, label_lst):
    dataset, labels = [], []

    for l, d in zip(label_lst, data_lst):
        data = pd.read_csv(base_path + SUB + "/" + d)
        data = data.iloc[:, 1:]   # 첫 column(time) 제거
        label = get_label_from_filename(l)
        labels.append(np.array([label] * data.shape[0]))
        dataset.append(data)
        #print("\t", l, d, " ====> ", np.array([label] * data.shape[0]).shape, data.shape)

    D = np.concatenate(dataset, axis=0)
    L = np.concatenate(labels, axis=0)
    #print(D.shape, L.shape)

    balanced_idx = modules.Downsample_to_balance_class(L)
    X_emg_bal, y_bal = D[balanced_idx], L[balanced_idx]
    print("\t Balanced EMG:", X_emg_bal.shape, "y:", y_bal.shape)

    num_channels = X_emg_bal.shape[1]

    X_tmp, y_tmp = processing.sliding_window_with_labels(X=X_emg_bal, y=y_bal,  window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

    all_X, all_y = modules.get_X_y_ZC_only(X_tmp, y_tmp)
    all_y = modules.y_change_to_int(all_y)
    all_X = all_X.reshape(-1, num_channels, 1)

    return all_X, all_y

def get_data(sub_info, data_lst, label_lst):
    dataset, labels = [], []

    for l, d in zip(label_lst, data_lst):
        data = pd.read_csv(base_path + sub_info + "/" + d)
        data = data.iloc[:, 1:]   # 첫 column(time) 제거
        label = get_label_from_filename(l)
        labels.append(np.array([label] * data.shape[0]))
        dataset.append(data)
        #print("\t", l, d, " ====> ", np.array([label] * data.shape[0]).shape, data.shape)

    D = np.concatenate(dataset, axis=0)
    L = np.concatenate(labels, axis=0)
    #print(D.shape, L.shape)

    balanced_idx = modules.Downsample_to_balance_class(L)
    X_emg_bal, y_bal = D[balanced_idx], L[balanced_idx]
    print("\t Balanced EMG:", X_emg_bal.shape, "y:", y_bal.shape)

    return X_emg_bal, y_bal


def sliding_window(X,y):
    num_channels = X.shape[1]
    X_tmp, y_tmp = processing.sliding_window_with_labels(X=X, y=y,  window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

    all_X, all_y = modules.get_X_y_ZC_only(X_tmp, y_tmp)
    all_y = modules.y_change_to_int(all_y)
    #all_X = all_X.reshape(-1, num_channels, 1)

    return all_X, all_y

In [2]:
classes = ['STC', 'WAK', 'STDUP', 'SITDN', 'UPS', 'DNS', 'KLFT', 'TPTO', 'LLF', 'LLB', 'LLS', 'KLCL', 'HS', 'TO', 'LUGF', 'LUGB'] #16 classes
class_to_label = {cls: i for i, cls in enumerate(classes)}

Features = ['mav', 'var', 'zc', 'iemg', 'wl', 'wamp', 'mavs', 'rms', 'ssc', 'msq', 'v3', 'ld', 'dabs', 'mfl', 'mpr', 'mnf', 'psr', 'arc1', 'arc2', 'arc3', 'arc4', 'cc1', 'cc2', 'cc3', 'cca', 'dwtc1', 'dwtc2', 'dwtpc1', 'dwtpc2', 'dwtpc3']

base_path = 'D:/Data/Gait-EMG/SIAT/SIAT_LLMD20230404/'
Sub_lst_data = [f"Sub{i:02d}"+"/Data/" for i in range(1, 41)]
Sub_lst_label = [f"Sub{i:02d}"+"/Labels/" for i in range(1, 41)]
window_size, step_size = 400, 20

# Data Store
- To reduce time so that I don't have to run the sliding window/ feature extraction everytime

In [ ]:
for SUB, LAB in zip(Sub_lst_data, Sub_lst_label):

    data_lst = os.listdir(base_path + SUB)
    label_lst = os.listdir(base_path + LAB)
    X_target, y_target = get_data(SUB, data_lst, label_lst)
    #X_target, y_target = modules.random_downsample_Num(X_target, y_target, n_keep=1000)
    print(f"Sub: {SUB} Start ==> (X, y:) {X_target.shape, y_target.shape} \n======================================================\n")
    X_store, y_store = sliding_window(X_target, y_target)
    pd.DataFrame(X_store).to_csv(f"./stored_dataset/X_{SUB[:-6]}.csv", index=False) #moved to D drive
    pd.DataFrame(y_store).to_csv(f"./stored_dataset/y_{SUB[:-6]}.csv", index=False)

# Baseline - LOSO

In [6]:
stored_data_path = 'C:/Users/hml76/PycharmProjects/Cross-motor-decoding/Data/Gait3/stored_dataset/'

ACC_lst = []
for SUB, LAB in zip(Sub_lst_data, Sub_lst_label):

    X_target = pd.read_csv(stored_data_path+f'X_{SUB[:-6]}.csv')
    y_target = pd.read_csv(stored_data_path+f'y_{SUB[:-6]}.csv')
    X_target, y_target = np.array(X_target), np.array(y_target)
    num_channels = X_target.shape[1]
    X_target = X_target.reshape(-1, num_channels, 1)

    X_target, y_target = modules.random_downsample_Num(X_target, y_target, n_keep=1000)
    print(f"Sub: {SUB} Start ==> (X, y:) {X_target.shape, y_target.shape} \n======================================================\n")

    X_lst, y_lst, Sub_acc = [], [], []
    for SUB2, LAB2 in zip(Sub_lst_data, Sub_lst_label):
        if SUB == SUB2:
            continue
        else:
            X = pd.read_csv(stored_data_path+f'X_{SUB2[:-6]}.csv')
            y = pd.read_csv(stored_data_path+f'y_{SUB2[:-6]}.csv')
            X, y = np.array(X), np.array(y)
            num_channels = X.shape[1]
            X = X.reshape(-1, num_channels, 1)
            X, y = modules.random_downsample_Num(X, y, n_keep=5000)

            print("\t Current - baseline : ", X.shape, y.shape)

            model = Model.build_model_1D(input_shape=X.shape[1:], num_classes=len(np.unique(y)))
            history = model.fit(X, y, epochs=50, batch_size=512, verbose=0)
            loss, acc = model.evaluate(X_target, y_target, verbose=0)
            print(f"\t Subject {SUB2} - inter subject acc => {acc*100:.2f}%")
            Sub_acc.append(acc)
            del model, history
    ACC_lst.append(Sub_acc)

Sub: Sub01/Data/ Start ==> (X, y:) ((1000, 25, 1), (1000, 1)) 

	 Current - baseline :  (5000, 25, 1) (5000, 1)
	 Subject Sub02/Data/ - inter subject acc => 13.60%
	 Current - baseline :  (5000, 25, 1) (5000, 1)
	 Subject Sub03/Data/ - inter subject acc => 8.60%
	 Current - baseline :  (5000, 25, 1) (5000, 1)
	 Subject Sub04/Data/ - inter subject acc => 14.60%
	 Current - baseline :  (5000, 25, 1) (5000, 1)
	 Subject Sub05/Data/ - inter subject acc => 21.00%
	 Current - baseline :  (5000, 25, 1) (5000, 1)
	 Subject Sub06/Data/ - inter subject acc => 5.20%
	 Current - baseline :  (5000, 25, 1) (5000, 1)
	 Subject Sub07/Data/ - inter subject acc => 33.80%
	 Current - baseline :  (5000, 25, 1) (5000, 1)
	 Subject Sub08/Data/ - inter subject acc => 12.80%
	 Current - baseline :  (5000, 25, 1) (5000, 1)
	 Subject Sub09/Data/ - inter subject acc => 9.60%
	 Current - baseline :  (5000, 25, 1) (5000, 1)
	 Subject Sub10/Data/ - inter subject acc => 9.70%
	 Current - baseline :  (5000, 25, 1) (5

In [7]:
ACC_lst

[[0.13600000739097595,
  0.0860000029206276,
  0.1459999978542328,
  0.20999999344348907,
  0.052000001072883606,
  0.33799999952316284,
  0.12800000607967377,
  0.09600000083446503,
  0.09700000286102295,
  0.12099999934434891,
  0.22499999403953552,
  0.23800000548362732,
  0.24500000476837158,
  0.18799999356269836,
  0.12099999934434891,
  0.04800000041723251,
  0.18299999833106995,
  0.16699999570846558,
  0.07800000160932541,
  0.054999999701976776,
  0.09700000286102295,
  0.23600000143051147,
  0.13699999451637268,
  0.2529999911785126,
  0.07900000363588333,
  0.17900000512599945,
  0.15800000727176666,
  0.09300000220537186,
  0.10300000011920929,
  0.06400000303983688,
  0.08299999684095383,
  0.11500000208616257,
  0.1379999965429306,
  0.17599999904632568,
  0.3619999885559082,
  0.019999999552965164,
  0.16899999976158142,
  0.09099999815225601,
  0.164000004529953],
 [0.20000000298023224,
  0.27000001072883606,
  0.20800000429153442,
  0.3269999921321869,
  0.30399999022

In [9]:
np.array(ACC_lst).shape

(40, 39)

In [11]:
sub_lst = [f"Sub{i:02d}" for i in range(1, 41)]
#pd.DataFrame(np.array(ACC_lst).T, columns=sub_lst).to_csv("./Gait3_LOSO_baseline.csv", index=False)

# Inter-subject : Accumulate

In [ ]:
stored_data_path = 'C:/Users/hml76/PycharmProjects/Cross-motor-decoding/Data/Gait3/stored_dataset/'

ACC_lst = []
for SUB, LAB in zip(Sub_lst_data[1:], Sub_lst_label[1:]):

    X_target = pd.read_csv(stored_data_path+f'X_{SUB[:-6]}.csv')
    y_target = pd.read_csv(stored_data_path+f'y_{SUB[:-6]}.csv')
    X_target, y_target = np.array(X_target), np.array(y_target)
    num_channels = X_target.shape[1]
    X_target = X_target.reshape(-1, num_channels, 1)

    X_target, y_target = modules.random_downsample_Num(X_target, y_target, n_keep=1000)
    print(f"Sub: {SUB} Start ==> (X, y:) {X_target.shape, y_target.shape} \n======================================================\n")

    X_lst, y_lst, Sub_acc = [], [], []
    for SUB2, LAB2 in zip(Sub_lst_data, Sub_lst_label):
        if SUB == SUB2:
            continue
        else:
            X_tmp = pd.read_csv(stored_data_path+f'X_{SUB2[:-6]}.csv')
            y_tmp = pd.read_csv(stored_data_path+f'y_{SUB2[:-6]}.csv')
            X_tmp, y_tmp = np.array(X_tmp), np.array(y_tmp)
            num_channels = X_tmp.shape[1]
            X_tmp = X_tmp.reshape(-1, num_channels, 1)

            X_tmp, y_tmp = modules.random_downsample_Num(X_tmp, y_tmp, n_keep=5000)

            X_lst.append(X_tmp)
            y_lst.append(y_tmp)
            X, y = np.concatenate(X_lst, axis=0), np.concatenate(y_lst, axis=0)

            print("\t Cumulated : ", X.shape, y.shape)

            model = Model.build_model_1D(input_shape=X.shape[1:], num_classes=len(np.unique(y)))
            history = model.fit(X, y, epochs=50, batch_size=512, verbose=0)
            loss, acc = model.evaluate(X_target, y_target, verbose=0)
            print(f"\t Subject {SUB2} - inter subject acc => {acc*100:.2f}%")
            Sub_acc.append(acc)
    ACC_lst.append(Sub_acc)

# Few-shot adaptation

In [5]:
stored_data_path = 'C:/Users/hml76/PycharmProjects/Cross-motor-decoding/Data/Gait3/stored_dataset/'
ACC_all_before, ACC_all_after = [], []
K = 10

for SUB, LAB in zip(Sub_lst_data, Sub_lst_label):

    X_train = pd.read_csv(stored_data_path+f'X_{SUB[:-6]}.csv')
    y_train = pd.read_csv(stored_data_path+f'y_{SUB[:-6]}.csv')
    X_train, y_train = np.array(X_train), np.array(y_train)
    num_channels = X_train.shape[1]
    X_train = X_train.reshape(-1, num_channels, 1)

    X_train, y_train = modules.random_downsample_Num(X_train, y_train, n_keep=1000)
    print(f"Sub: {SUB} Start ==> (X, y:) {X_train.shape, y_train.shape} \n======================================================\n")
    Sub_acc_before, Sub_acc_after = [], []

    for SUB2, LAB2 in zip(Sub_lst_data, Sub_lst_label):
        if SUB == SUB2:
            continue
        else:
            X = pd.read_csv(stored_data_path+f'X_{SUB2[:-6]}.csv')
            y = pd.read_csv(stored_data_path+f'y_{SUB2[:-6]}.csv')
            X, y = np.array(X), np.array(y)
            num_channels = X.shape[1]
            X = X.reshape(-1, num_channels, 1)
            X, y = modules.random_downsample_Num(X, y, n_keep=5000)
            print("\t Cumulated : ", X.shape, y.shape)

            model = Model.build_model_1D(input_shape=X_train.shape[1:], num_classes=len(np.unique(y_train)))
            history = model.fit(X_train, y_train, epochs=50, batch_size=512, verbose=0)
            meta = Adaptation.MetaLearner(input_model=model, N_way=len(np.unique(y_train)), input_shape=X_train.shape[1:],
                        meta_iters=5,                             # Number of meta-training loops
                        meta_step_size=0.02                          # Reptile meta step
                    )
            X_train_meta, X_test, y_train_meta, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            loss, acc_before = model.evaluate(X_test, y_test, verbose=0)
            acc_after = meta.train(X_train_meta, y_train_meta, X_test, y_test, Adaptation.get_data_Meta, N_way=len(np.unique(y_test)), K_shot=K)

            print(f"\t Target sub {SUB2[:-6]} acc => before: {acc_before*100:.2f}% | after: {acc_after*100:.2f}%")
            Sub_acc_before.append(acc_before)
            Sub_acc_after.append(acc_after)

    print("\n\n")
    ACC_all_before.append(Sub_acc_before)
    ACC_all_after.append(Sub_acc_after)

Sub: Sub01/Data/ Start ==> (X, y:) ((1000, 25, 1), (1000, 1)) 

	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub02 acc => before: 18.80% | after: 63.40%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub03 acc => before: 16.20% | after: 67.30%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub04 acc => before: 12.50% | after: 64.90%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub05 acc => before: 17.00% | after: 60.60%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub06 acc => before: 12.10% | after: 75.40%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub07 acc => before: 33.30% | after: 60.30%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub08 acc => before: 16.30% | after: 36.60%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub09 acc => before: 12.10% | after: 44.60%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub10 acc => before: 12.40% | after: 62.60%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub11 acc => before:

In [6]:
pd.DataFrame(ACC_all_before).shape

(40, 39)

In [7]:
sub_lst = [f"Sub{i:02d}" for i in range(1, 41)]
pd.DataFrame(np.array(ACC_all_before).T, columns=sub_lst).to_csv(f'./Leave_one_subject_out_few_shot_K{K}_before.csv', index=False)
pd.DataFrame(np.array(ACC_all_after).T, columns=sub_lst).to_csv(f'./Leave_one_subject_out_few_shot_K{K}_after.csv', index=False)

In [5]:
def K_shot_run(K):
    stored_data_path = 'C:/Users/hml76/PycharmProjects/Cross-motor-decoding/Data/Gait3/stored_dataset/'
    ACC_all_before, ACC_all_after = [], []

    for SUB, LAB in zip(Sub_lst_data, Sub_lst_label):

        X_train = pd.read_csv(stored_data_path+f'X_{SUB[:-6]}.csv')
        y_train = pd.read_csv(stored_data_path+f'y_{SUB[:-6]}.csv')
        X_train, y_train = np.array(X_train), np.array(y_train)
        num_channels = X_train.shape[1]
        X_train = X_train.reshape(-1, num_channels, 1)

        X_train, y_train = modules.random_downsample_Num(X_train, y_train, n_keep=1000)
        print(f"Sub: {SUB} Start ==> (X, y:) {X_train.shape, y_train.shape} \n======================================================\n")
        Sub_acc_before, Sub_acc_after = [], []

        for SUB2, LAB2 in zip(Sub_lst_data, Sub_lst_label):
            if SUB == SUB2:
                continue
            else:
                X = pd.read_csv(stored_data_path+f'X_{SUB2[:-6]}.csv')
                y = pd.read_csv(stored_data_path+f'y_{SUB2[:-6]}.csv')
                X, y = np.array(X), np.array(y)
                num_channels = X.shape[1]
                X = X.reshape(-1, num_channels, 1)
                X, y = modules.random_downsample_Num(X, y, n_keep=5000)
                print("\t Cumulated : ", X.shape, y.shape)

                model = Model.build_model_1D(input_shape=X_train.shape[1:], num_classes=len(np.unique(y_train)))
                history = model.fit(X_train, y_train, epochs=50, batch_size=512, verbose=0)
                meta = Adaptation.MetaLearner(input_model=model, N_way=len(np.unique(y_train)), input_shape=X_train.shape[1:],
                            meta_iters=5,                             # Number of meta-training loops
                            meta_step_size=0.02                          # Reptile meta step
                        )
                X_train_meta, X_test, y_train_meta, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                loss, acc_before = model.evaluate(X_test, y_test, verbose=0)
                acc_after = meta.train(X_train_meta, y_train_meta, X_test, y_test, Adaptation.get_data_Meta, N_way=len(np.unique(y_test)), K_shot=K)

                print(f"\t Target sub {SUB2[:-6]} acc => before: {acc_before*100:.2f}% | after: {acc_after*100:.2f}%")
                Sub_acc_before.append(acc_before)
                Sub_acc_after.append(acc_after)

        print("\n\n")
        ACC_all_before.append(Sub_acc_before)
        ACC_all_after.append(Sub_acc_after)
    try:
        sub_lst = [f"Sub{i:02d}" for i in range(1, 41)]
        pd.DataFrame(np.array(ACC_all_before).T, columns=sub_lst).to_csv(f'./Leave_one_subject_out_few_shot_K{K}_before.csv', index=False)
        pd.DataFrame(np.array(ACC_all_after).T, columns=sub_lst).to_csv(f'./Leave_one_subject_out_few_shot_K{K}_after.csv', index=False)
    except:
        pass

    return ACC_all_before, ACC_all_after


In [6]:
 a,b = K_shot_run(K=2)

Sub: Sub01/Data/ Start ==> (X, y:) ((1000, 25, 1), (1000, 1)) 

	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub02 acc => before: 30.00% | after: 48.20%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub03 acc => before: 20.30% | after: 29.00%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub04 acc => before: 25.70% | after: 28.50%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub05 acc => before: 15.80% | after: 29.70%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub06 acc => before: 13.90% | after: 15.50%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub07 acc => before: 19.50% | after: 24.80%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub08 acc => before: 20.70% | after: 43.30%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub09 acc => before: 17.10% | after: 21.20%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub10 acc => before: 10.40% | after: 29.60%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub11 acc => before:

In [7]:
 a1,b1 = K_shot_run(K=1)

Sub: Sub01/Data/ Start ==> (X, y:) ((1000, 25, 1), (1000, 1)) 

	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub02 acc => before: 29.50% | after: 58.90%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub03 acc => before: 14.60% | after: 26.00%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub04 acc => before: 21.30% | after: 41.20%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub05 acc => before: 20.90% | after: 35.70%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub06 acc => before: 13.60% | after: 21.40%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub07 acc => before: 24.40% | after: 28.40%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub08 acc => before: 20.10% | after: 33.10%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub09 acc => before: 11.80% | after: 33.10%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub10 acc => before: 8.90% | after: 22.30%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub11 acc => before: 

In [8]:
 a5,b5 = K_shot_run(K=5)

Sub: Sub01/Data/ Start ==> (X, y:) ((1000, 25, 1), (1000, 1)) 

	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub02 acc => before: 21.40% | after: 60.30%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub03 acc => before: 17.80% | after: 37.30%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub04 acc => before: 21.20% | after: 41.60%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub05 acc => before: 10.40% | after: 29.40%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub06 acc => before: 14.50% | after: 33.90%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub07 acc => before: 32.30% | after: 53.50%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub08 acc => before: 12.50% | after: 31.70%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub09 acc => before: 0.00% | after: 46.60%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub10 acc => before: 9.00% | after: 44.20%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub11 acc => before: 1

In [9]:
 a20,b20 = K_shot_run(K=20)

Sub: Sub01/Data/ Start ==> (X, y:) ((1000, 25, 1), (1000, 1)) 

	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub02 acc => before: 22.30% | after: 75.60%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub03 acc => before: 12.40% | after: 68.90%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub04 acc => before: 24.20% | after: 84.20%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub05 acc => before: 19.00% | after: 92.40%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub06 acc => before: 16.90% | after: 52.20%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub07 acc => before: 33.40% | after: 97.60%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub08 acc => before: 21.20% | after: 84.90%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub09 acc => before: 29.40% | after: 64.90%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub10 acc => before: 19.00% | after: 94.80%
	 Cumulated :  (5000, 25, 1) (5000, 1)
	 Target sub Sub11 acc => before: